In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [21]:
trainSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.train.bio')
# devSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testa.bio')
# testSentences = utils.get_sentences_conll('../data/CONLL/deu/deu_utf.testb.bio')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [22]:
print(testSentences[3])

[['Die', 'O'], ['These', 'O'], [',', 'O'], ['Schlatter', 'B-PER'], ['sei', 'O'], ['Antisemit', 'O'], ['gewesen', 'O'], [',', 'O'], ['wurde', 'O'], ['seither', 'O'], ['in', 'O'], ['der', 'O'], ['theologischen', 'O'], ['Fachliteratur', 'O'], ['nicht', 'O'], ['mehr', 'O'], ['vertreten', 'O'], ['.', 'O']]


In [23]:
labelSet = set()
characters= set()
models.max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)

In [24]:
print(len(labelSet))
print(models.max_sequence_length)

25
56


In [25]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [26]:
print(models.label2Idx)

{'I-ORGpart': 1, 'I-PER': 2, 'B-LOCderiv': 15, 'B-LOC': 14, 'B-PER': 3, 'I-LOCderiv': 4, 'I-LOC': 6, 'I-LOCpart': 17, 'B-PERderiv': 8, 'B-OTHderiv': 18, 'B-PERpart': 10, 'B-LOCpart': 20, 'B-OTH': 21, 'I-OTHderiv': 5, 'PADDING_TOKEN': 0, 'I-ORGderiv': 9, 'B-OTHpart': 16, 'I-ORG': 7, 'B-ORGpart': 23, 'O': 11, 'B-ORG': 24, 'I-OTH': 25, 'B-ORGderiv': 12, 'I-PERderiv': 22, 'I-PERpart': 13, 'I-OTHpart': 19}


In [27]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [28]:
print(models.case2Idx)

{'mainly_numeric': 6, 'initialUpper': 4, 'contains_digit': 7, 'allUpper': 3, 'numeric': 1, 'other': 5, 'PADDING_TOKEN': 0, 'allLower': 2}


In [29]:
print(type(trainSentences))

<class 'list'>


In [30]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [31]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'⋅': 5, 'ą': 6, 'X': 319, 'ē': 300, '_': 8, '−': 187, 'η': 9, 'λ': 304, '<': 56, 'ḫ': 10, '傳': 11, 'ō': 12, 'ř': 13, 'F': 14, 'š': 15, 'E': 16, '학': 17, '‹': 18, 'y': 19, 'Λ': 21, 'n': 22, 'K': 23, 'B': 112, 'S': 25, 'ḳ': 171, 'з': 26, '»': 27, 'ž': 28, 'r': 29, '</S>': 2, 'È': 32, '術': 225, '章': 275, 'v': 33, 'Ö': 34, 'П': 36, '→': 192, 'À': 37, 'Л': 38, '佐': 173, 'o': 39, 'ю': 40, '1': 41, 'Ø': 43, '`': 44, 'Ż': 47, 'č': 46, 'ā': 302, '}': 48, 'Å': 49, '³': 51, '"': 61, 'ъ': 327, 'ő': 52, 'µ': 53, 'п': 180, 'ñ': 55, 'ب': 57, 'у': 58, 'k': 177, 'д': 59, 'Ä': 179, '0': 60, '<W>': 3, '†': 62, 'ť': 63, 'ن': 65, 'ĩ': 66, 'J': 67, '\u200e': 69, 'ệ': 71, 'P': 72, 'õ': 75, ')': 74, 'υ': 77, 'В': 289, 'М': 282, 'т': 64, 'ж': 20, '別': 196, '©': 78, '≘': 80, '公': 83, 'z': 325, ':': 84, '½': 86, 'й': 88, 'à': 90, '寝': 284, '동': 91, '–': 92, '.': 93, 'к': 94, 'q': 95, 'γ': 97, '\xad': 98, 'ċ': 235, '$': 99, 'м': 100, 'Ш': 101, ';': 102, 'е': 125, 'ά': 70, '柯': 104, 'N': 105, 'φ': 107, '鶴': 109, 

In [32]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [34]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [35]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [36]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

26
26


# Test Model

In [19]:
# import importlib
# importlib.reload(models)

<module 'models' from '/home/gwiedemann/microNER/scripts/models.py'>

In [ ]:
tmp_model_filename = 'model_lstm2_germeval.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_2lstm_v2()

In [20]:
# import importlib
# importlib.reload(utils)

<module 'utils' from '/home/gwiedemann/microNER/scripts/utils.py'>

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences[:1000], shuffle_data=True, batch_size=256), 
    validation_data = utils.NerSequence(devSentences[:1000], batch_size=256), 
    epochs = 3, callbacks = [history]
)

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 5, callbacks = [history]
)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences[:1000])
print(compute_f1(pred_labels, true_labels, models.idx2Label))

# Experiments

In [ ]:
f = open('results_lstm2_germeval.txt', 'a')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'model_lstm2_germeval.' + str(run_i) + '.h5'

    history = utils.F1History(tmp_model_filename, devSet=devSentences)

    model = models.get_model_2lstm_v2()
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=16), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=256), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    true_labels, pred_labels = utils.predict_sequences(model, testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10688       char_input[0][0]                 
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, None, 52, 100 33200       char_embedding[0][0]             
__________________________________________________________________________________________________
word

Epoch 1/10
1499/1500 [============================>.] - ETA: 0s - loss: 0.0467 - acc: 0.9835
New maximum F1 score: 0.7574827321565617 (before: 0) Saving to model_lstm2_germeval.1.h5
1500/1500 [==============================] - 799s 533ms/step - loss: 0.0467 - acc: 0.9835 - val_loss: 0.0046 - val_acc: 0.9890
Epoch 2/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.0166 - acc: 0.9887
New maximum F1 score: 0.7847957639939486 (before: 0.7574827321565617) Saving to model_lstm2_germeval.1.h5
1500/1500 [==============================] - 788s 525ms/step - loss: -0.0166 - acc: 0.9887 - val_loss: -0.0443 - val_acc: 0.9903
Epoch 3/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.0697 - acc: 0.9901
New maximum F1 score: 0.7965433026488822 (before: 0.7847957639939486) Saving to model_lstm2_germeval.1.h5
1500/1500 [==============================] - 787s 525ms/step - loss: -0.0697 - acc: 0.9901 - val_loss: -0.0970 - val_acc: 0.9908
Epoch 4/10
1499/1500 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1499/1500 [============================>.] - ETA: 0s - loss: -0.1294 - acc: 0.9908
New maximum F1 score: 0.8047226386806596 (before: 0.8028009084027252) Saving to model_lstm2_germeval.2.h5
1500/1500 [==============================] - 753s 502ms/step - loss: -0.1294 - acc: 0.9908 - val_loss: -0.1558 - val_acc: 0.9905
Epoch 5/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.1848 - acc: 0.9915
New maximum F1 score: 0.8162273322116285 (before: 0.8047226386806596) Saving to model_lstm2_germeval.2.h5
1500/1500 [==============================] - 748s 499ms/step - loss: -0.1848 - acc: 0.9915 - val_loss: -0.2098 - val_acc: 0.9912
Epoch 6/10
1459/1500 [============================>.] - ETA: 19s - loss: -0.2392 - acc: 0.9922

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1500/1500 [==============================] - 752s 501ms/step - loss: -0.4036 - acc: 0.9932 - val_loss: -0.4239 - val_acc: 0.9911
Epoch 10/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.4577 - acc: 0.9935
New maximum F1 score: 0.8191391095510807 (before: 0.8162273322116285) Saving to model_lstm2_germeval.2.h5
1500/1500 [==============================] - 746s 497ms/step - loss: -0.4577 - acc: 0.9935 - val_loss: -0.4773 - val_acc: 0.9914
Epoch 1/10
39/94 [===========>..................] - ETA: 53s - loss: -0.4872 - acc: 0.9945

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1499/1500 [============================>.] - ETA: 0s - loss: -0.0161 - acc: 0.9888
New maximum F1 score: 0.7884542347132549 (before: 0.7645024971187091) Saving to model_lstm2_germeval.3.h5
1500/1500 [==============================] - 750s 500ms/step - loss: -0.0161 - acc: 0.9888 - val_loss: -0.0442 - val_acc: 0.9903
Epoch 3/10
 180/1500 [==>...........................] - ETA: 10:45 - loss: -0.0473 - acc: 0.9900

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_3cnn_bi-lstm.h5', '../models/final_model_germeval_outer.h5')
with open("../models/final_model_germeval.indexes", "w") as f:
    json.dump([models.idx2Label, models.label2Idx, models.char2Idx, models.case2Idx], f)

In [ ]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model_germeval_outer.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
f = open('germeval_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = "\t".join([str(i_tok+1), tok[0], correctlabel, correctlabel, guessedlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close